In [2]:
import cv2
import mediapipe as mp
import math

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Key body landmarks for pose estimation and exercise tracking
landmarks_to_track = {
    "nose": mp_pose.PoseLandmark.NOSE,
    "left_shoulder": mp_pose.PoseLandmark.LEFT_SHOULDER,
    "right_shoulder": mp_pose.PoseLandmark.RIGHT_SHOULDER,
    "left_elbow": mp_pose.PoseLandmark.LEFT_ELBOW,
    "right_elbow": mp_pose.PoseLandmark.RIGHT_ELBOW,
    "left_wrist": mp_pose.PoseLandmark.LEFT_WRIST,
    "right_wrist": mp_pose.PoseLandmark.RIGHT_WRIST,
    "left_hip": mp_pose.PoseLandmark.LEFT_HIP,
    "right_hip": mp_pose.PoseLandmark.RIGHT_HIP,
    "left_knee": mp_pose.PoseLandmark.LEFT_KNEE,
    "right_knee": mp_pose.PoseLandmark.RIGHT_KNEE,
    "left_ankle": mp_pose.PoseLandmark.LEFT_ANKLE,
    "right_ankle": mp_pose.PoseLandmark.RIGHT_ANKLE,
}

# Function to calculate angle between three landmarks
def calculate_angle(landmark_list, landmark1, landmark2, landmark3):
    x1, y1 = landmark_list[landmark1].x, landmark_list[landmark1].y
    x2, y2 = landmark_list[landmark2].x, landmark_list[landmark2].y
    x3, y3 = landmark_list[landmark3].x, landmark_list[landmark3].y

    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))

    if angle < 0:
        angle += 360

    return angle

# Function to track squats
def track_squats(landmark_list):
    knee_angle = calculate_angle(landmark_list, "left_knee", "left_hip", "left_ankle")
    hip_angle = calculate_angle(landmark_list, "left_shoulder", "left_hip", "left_knee")

    squat_threshold_knee = 120  # Adjust based on your flexibility
    squat_threshold_hip = 90  # Adjust based on your form

    # Check if both knee and hip angles meet squat criteria (can be adjusted for variations)
    if knee_angle > squat_threshold_knee and hip_angle > squat_threshold_hip:
        return True
    else:
        return False

# Function to track push-ups
def track_pushups(landmark_list):
    elbow_angle = calculate_angle(landmark_list, "left_shoulder", "left_elbow", "left_wrist")
    plank_angle = calculate_angle(landmark_list, "left_shoulder", "left_hip", "left_knee")

    pushup_threshold_elbow = 90  # Adjust based on your push-up form
    plank_threshold = 135  # Adjust based on your body position

    # Check if both elbow and plank angles meet push-up criteria (can be adjusted for variations)
    if elbow_angle <= pushup_threshold_elbow and plank_angle >= plank_threshold:
        return True
    else:
        return False

def main():
    # Initialize video capture
    cap = cv2.VideoCapture(0)

    # Create a Pose object
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        squat_count = 0
        pushup_count = 0
        squat_detected = False
        pushup_detected = False
        while cap.isOpened():
            success, image = cap.read()

            if not success:
                print("Ignoring empty camera frame.")
                continue

            # Convert the BGR image to RGB for pose estimation
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Flip the image horizontally for a later selfie-view display
            image_rgb = cv2.flip(image_rgb, 1)

            # Detect poses in the frame
            results = pose.process(image_rgb)

            if results.pose_landmarks:
                # Draw pose landmarks on the image
                mp_drawing.draw_landmarks(
                    image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                )

                # Create a dictionary to store landmark points
                landmark_points = {}
                for landmark, landmark_name in landmarks_to_track.items():
                    if results.pose_landmarks.landmark[landmark_name]:
                        landmark_points[landmark] = results.pose_landmarks.landmark[landmark_name]

                # Track squats and push-ups
                if not squat_detected and track_squats(landmark_points):
                    squat_detected = True
                    squat_count += 1
                elif squat_detected and not track_squats(landmark_points):
                    squat_detected = False
                
                if not pushup_detected and track_pushups(landmark_points):
                    pushup_detected = True
                    pushup_count += 1
                elif pushup_detected and not track_pushups(landmark_points):
                    pushup_detected = False

                # Display squat and push-up counts
                cv2.putText(
                    image,
                    f"Squats: {squat_count}",
                    (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1,
                    (0, 255, 0),
                    2,
                )
                cv2.putText(
                    image,
                    f"Push-ups: {pushup_count}",
                    (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1,
                    (0, 255, 0),
                    2,
                )

            # Display the resulting frame
            cv2.imshow("Pose Tracking", image)

            # Break the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break

        # Release the video capture object and close all windows
        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
